In [43]:
import requests
import re
import json
from tqdm import tqdm

# Metacritic API url
url = "https://mcgqlapi.com/api"

# Using readlines()
albums_file = open('albums.txt', 'r')
albums_lines = albums_file.readlines()

albums = {}

# Obtain information using API
for album_line in albums_lines:
    #elements = [e.strip() for e in album_line.split("-")]
    elements = [e.strip() for e in album_line.split(" - ")]
    try:
        artist, genre = elements[1].split("(")[0].strip(), elements[1].split("(")[1].replace(")", "").strip()
        # criar um dicionário com as informações obtidas
        print(elements[0])
        print(elements[4])
        album = {
            "title": elements[0],
            "artist": artist,
            "genre": genre,
            "year": int(elements[2]),
            "label": elements[3],
            "sales": elements[4]
        }

        # adicionar o dicionário ao dicionário principal
        albums[elements[0]] = {"data": {"album": album}}

    except IndexError:
        # Skip malformed line
        continue

with open('albums.json', 'w') as output_file:
    output_file.write(json.dumps(albums))

Thriller
66 million
The Dark Side of the Moon
45 million
Back in Black
50 million
The Bodyguard Soundtrack
45 million
Bat Out of Hell
43 million
Rumours
45 million
Their Greatest Hits (1971-1975)
42 million
Saturday Night Fever Soundtrack
40 million
Appetite for Destruction
30 million
The Joshua Tree
25 million
21
31 million
Come On Over
40 million
The Eminem Show
30 million
The Beatles (The White Album)
30 million
Born in the U.S.A.
30 million
Brothers in Arms
30 million
Millennium
24 million
Legend
28 million
Jagged Little Pill
33 million
Hotel California
32 million
Like a Virgin
25 million
Goodbye Yellow Brick Road
31 million
Hysteria
25 million
1989
20 million
Faith
25 million
Purple Rain
25 million
True Blue
25 million
Physical Graffiti
22 million
No Jacket Required
25 million
1984
20 million
The Very Best of The Eagles
25 million
Greatest Hits
25 million
Jagged Little Pill Acoustic
20 million
Brothers and Sisters
22 million
Tapestry
25 million
Tragic Kingdom
20 million
Bridge ove

In [44]:
import json
import pandas as pd
import csv


# Abra o arquivo JSON e carregue o conteúdo em uma variável
with open('albums.json', 'r') as file:
    data = json.load(file)

with open('albums.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'artist', 'genre', 'year', 'label', 'sales'])
    
    for album_title, album_data in data.items():
        album = album_data['data']
        if((album.get('album'))):
            writer.writerow([
                album['album']['title'],
                album['album']['artist'],
                album['album']['genre'],
                album['album']['year'],
                album['album']['label'],
                album['album']['sales']
            ])

In [45]:
pd.read_csv("albums.csv")

,title,artist,genre,year,label,sales
0,Thriller,Michael Jackson,Pop/R&B,1982,Epic Records,110 million
1,The Dark Side of the Moon,Pink Floyd,Progressive Rock,1973,Harvest Records,50 million
2,Back in Black,AC/DC,Hard Rock,1980,Atlantic Records,50 million
3,The Bodyguard Soundtrack,Various Artists,Soundtrack/Pop,1992,Arista Records,45 million
4,Bat Out of Hell,Meat Loaf,Rock,1977,Epic Records,43 million
...,...,...,...,...,...,...
96,You Forgot It in People,Broken Social Scene,Indie Rock,2002,Arts & Crafts,"100,000"
97,Let It Come Down,Spiritualized,Space Rock,2001,Arista Records,"100,000"
98,The Argument,Fugazi,Post-Hardcore,2001,Dischord Records,"100,000"
99,Person Pitch,Panda Bear,Experimental Pop,2007,Paw Tracks,"100,000"
